# Creating a custom Index in Pandas

The pandas.Datetimeindex is the jam because of the extra properties that access portions of the time.  I'd really like a `pathlib` version of this, `...` and a `luigi` target version.

In [3]:
from requests import *from pandas import Index, Series, DataFrame, np; import pandas as pd
from toolz.curried.operator import *; from toolz.curried import *

In [92]:
import pickle

In [60]:
from functools import wraps

class BaseIndex(Index):
    _base, _dtype = Index, object
    
    @property
    def _attr_mapper(self):
        yield tuple(dir(self._base)), Series
    
    def __new__(cls, data=None, dtype=None, copy=False, name=None, fastpath=False, tupleize_cols=True, **kwargs):
        if isiterable(data) and not isinstance(data, Index):
            try:
                data = cls._coerce_to_ndarray(map(cls._base, data))
            except (TypeError, ValueError): ...
            
        return cls._simple_new(data, name, dtype=dtype, copy=copy, fastpath=fastpath, tupleize_cols=tupleize_cols, **kwargs).astype(dtype or cls._dtype)
    
    def __getattribute__(self, name):
        try:
            return super().__getattribute__(name)
        except AttributeError as e:
            for attrs, cls in self._attr_mapper:
                if name in attrs: break
            else:
                raise e
            
            attr = getattr(self._base, name, None)
            
            # There is no way to interpret the attribute
            if cls is False: raise NotImplemented
            
            if isinstance(attr, property) or not callable(attr): #  return computed values, not a callable.
                result = cls(self.map(attrgetter(name)))
                if cls in (Series, DataFrame): 
                    result.index = self
                return (self if cls is None else result)

            # wrap a callable for later.
            @wraps(attr)
            def mapper(*args, **kwargs):                        
                if cls is None: return self        
                result = cls(self.map(partial(lambda v: attr(v, *args, **kwargs))))
                if cls in (Series, DataFrame): result.index = self
                return result
            return mapper
    
    def __dir__(self):
        """Update the available attribute."""
        return list(super().__dir__()) + dir(self._base) + list(concat(pluck(0, self._attr_mapper)))
    
    def astype(self, dtype, copy=True):
        if dtype: self._data = self._data.astype(dtype, copy=copy)
        return self

In [61]:
from requests import *

In [163]:
_cache = dict()

import requests

class RequestIndex(BaseIndex):
    _base = str
    def get(self, **kwargs):
        return ResponseIndex([memoize(
            cache=_cache, key=lambda a, k: pickle.dumps((a,k))
        )(requests.get)(value, **kwargs) for value in self])
    
    def __add__(self, value, prefix=""):
        if not isinstance(value, str) and isiterable(value):
            return type(self)(str(_0)+prefix+str(_1) for _0, _1 in zip(self, value))
        return self.map(flip(add)(prefix+str(value)))
    
    def __truediv__(self, value):
        return self.__add__(value, '/')

In [164]:
import requests

class ResponseIndex(BaseIndex):
    _base = Response
    _attr_mapper = [tuple(Response.__dict__.keys()), Series], [tuple(Response.__attrs__), Index]

In [165]:
i = RequestIndex(['http://localhost:8888/api', 'http://localhost:8888/api/contents'])

v = i.get(params={'foo': 'bar'}).text

In [166]:
s = RequestIndex(['https://api.github.com/users/tonyfast']).get()

In [174]:
gists = Series([
    s.json()[0]['gists_url'].replace('{/gist_id}', '?page=')
]*2).reset_index().pipe(lambda df: RequestIndex(df[0])+(df.index+1)).get().json()

In [177]:
pd.concat(dict(zip(gists.index.url, gists.apply(DataFrame).values)))

comments  \
https://api.github.com/users/tonyfast/gists?page=1 0          0   
                                                   1          0   
                                                   2          0   
                                                   3          0   
                                                   4          0   
                                                   5          0   
                                                   6          0   
                                                   7          0   
                                                   8          0   
                                                   9          0   
                                                   10         0   
                                                   11         0   
                                                   12         0   
                                                   13         0   
                                                   14         0   
                                                   15         0   
                                                   16         0   
                                                   17         0   
                                                   18         0   
                                                   19         0   
                                                   20         0   
                                                   21         0   
                                                   22         2   
                                                   23         0   
                                                   24         0   
                                                   25         0   
                                                   26         1   
                                                   27         0   
                                                   28         0   
                                                   29         0   
https://api.github.com/users/tonyfast/gists?page=2 0          0   
                                                   1          0   
                                                   2          0   
                                                   3          0   
                                                   4          0   
                                                   5          0   
                                                   6          0   
                                                   7          0   
                                                   8          0   
                                                   9          0   
                                                   10         0   
                                                   11         0   
                                                   12         0   
                                                   13         0   
                                                   14         0   
                                                   15         0   
                                                   16         0   
                                                   17         0   
                                                   18         0   
                                                   19         0   
                                                   20         0   
                                                   21         0   
                                                   22         0   
                                                   23         0   
                                                   24         0   
                                                   25         0   
                                                   26         0   
                                                   27         0   
                                                   28         0   
                                   

In [172]:
gists = Q

RequestIndex(['https://api.github.com/users/tonyfast/gists?page=1', 'https://api.github.com/users/tonyfast/gists?page=2'], dtype='object')

In [178]:
from pathlib import Path

In [179]:
class PathIndex(BaseIndex):
    _base = Path
    
    def __truediv__(self, value):
        return type(self)(self.map(flip(Path.__truediv__)(value)))
    
    def __rtruediv__(self, value):
        return type(self)(self.map(flip(Path.__rtruediv__)(value)))
    
PathIndex._attr_mapper = [
    [['glob', 'rglob', 'iterdir'], compose(PathIndex, list, concat)],
    [tuple(dir(Path)), Series]
]


In [99]:
p = PathIndex(['.'])